<!-- # Processing the final dataset
 -->


# Export data as CSV

In [2]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year

## Data for all biomass and age data comparisons

In [ ]:
biome =  ee.Image(f"{data_folder}/categorical").select("biome")

# Ages
mapbiomas = ee.Image(f"{data_folder}/mapbiomas_{last_year}").rename(["age_mapbiomas"])
tmf = ee.Image(f"{data_folder}/tmf_{last_year}").rename(["age_tmf"])
silva = ee.Image("projects/ee-regrowth/assets/MB_secondary_forest_Silva_Junior_v2/sforestAge_brazil_V8").select("classification_2020").rename("age_silva")

# Response variables
GEDI_L2A = ee.Image(f"{data_folder}/GEDI_L2A_{last_year}")
GEDI_L4A = ee.Image(f"{data_folder}/GEDI_L4A_{last_year}")
ESA_CCI = (ee.Image(f"projects/sat-io/open-datasets/ESA/ESA_CCI_AGB/CCI_BIOMASS_100m_AGB_{last_year}_v51")
           .select("AGB").rename(f"ESA_CCI_{last_year}"))
heinrich = (ee.Image("projects/ee-regrowth/assets/Heinrich_etal_2021_updates/sforestAGC_climate_only_v1_1")
            .select("classification_2020").rename("heinrich_biomass_2020"))

# Combine all images into a single multi-band image
comparisons = mapbiomas.addBands([tmf, silva, ESA_CCI, GEDI_L2A, GEDI_L4A, heinrich, biome])
mask = comparisons.reduce(ee.Reducer.allNonZero())
masked_image = comparisons.updateMask(mask)

comparisons_sampled = comparisons.stratifiedSample(numPoints = 10000, classBand = 'biome')

export_csv(comparisons_sampled, "comparisons_sampled")

## Data for mature forest biomass comparisons and distance to edge plot

In [ ]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass").addBands(biome)
mature_biomass_sampled = mature_biomass.stratifiedSample(numPoints = 10000, classBand = 'biome')
export_csv(mature_biomass_sampled, "mature_biomass_sampled")

## Data for field data comparisons

In [ ]:
field_data = ee.FeatureCollection(f"{data_folder}/field_biomass") # from https://github.com/forc-db/GROA/tree/master/data
biomes = ee.FeatureCollection(f"{data_folder}/raw/biomes_br").select('CD_Bioma')

# Check in which biome each field plot is located
def determine_biome(feature):
    bioma_number = biomes.filterBounds(feature.geometry()).first().get('CD_Bioma')
    return feature.set('biome', bioma_number)

field_biomass = field_data.map(determine_biome)

export_csv(field_biomass, "field_biomass")

# Main Model Dataset

In [ ]:


categorical = ee.Image(f"{data_folder}/categorical")


land_use = ee.Image(f"{data_folder}/mapbiomas/land_use_no_fire")

distance_to_nearest_mature = ee.Image(f"{data_folder}/distance_to_nearest_mature")

cwd = ee.Image(f"{data_folder}/raw/cwd_chave").int16()

sur_cover = ee.Image(f"{data_folder}/sur_cover")

topography = ee.Image("CSP/ERGo/1_0/Global/ALOS_landforms").rename("topography") # 90m resolution

soil = ee.Image(f"{data_folder}/soilgrids")

climate = ee.Image(f"{data_folder}/yearly_terraclim").select( # 10,000m resolution
 'mean_pr',
 'mean_srad',
 'mean_temp',
 'mean_vpd',
 'mean_soil',
 'mean_aet',
 'mean_si')

nearest_mature = ee.Image(f"{data_folder}/mapbiomas/nearest_mature_biomass_image_neighborhood").rename("nearest_mature_biomass")

quarters_ecoreg_biomass = ee.Image("projects/amazon-forest-regrowth/assets/quarters_ecoreg_biomass")


def keep_rare_lu_types(unified_data):
    # Define the list of bands to check
    bands_to_check = ['lulc_sum_20', 'lulc_sum_21', 'lulc_sum_35', 
                    'lulc_sum_39', 'lulc_sum_40', 'lulc_sum_41', 
                    'lulc_sum_46', 'lulc_sum_48', 'lulc_sum_9']

    # Create a mask where at least one of the specified bands is non-zero
    mask = unified_data.select(bands_to_check).reduce(ee.Reducer.anyNonZero())

    # # Apply the mask to the unified_data
    return unified_data.updateMask(mask)


age.pixelLonLat().float()

floodable_forests = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
        .select(f"classification_{last_year}").eq(6))


fire = (ee.Image("projects/mapbiomas-public/assets/brazil/fire/collection3/mapbiomas_fire_collection3_annual_burned_coverage_v1")
    .select([f"burned_coverage_{year}" for year in config.range_1985_2020])
    .byte()
    .rename([str(year) for year in config.range_1985_2020])
    .gt(0)
    .reduce('sum').rename("num_fires")).unmask(0)



In [ ]:

# Combine all bands with additional masking conditions
combined_mask = biomass.mask().And(distance_to_border_mask).And(one_hectare_mask).And(ages_mask).And(distance_to_nearest_mature).And(nearest_mature).And(fire.lt(14)).And(land_use.select("last_LU").mask())

unified_data = mapbiomas_age.addBands([silva_age, fire, heinrich_biomass_2020, biomass, categorical, nearest_mature, land_use,
                                       distance_to_nearest_mature, cwd, sur_cover, topography, soil, climate, amazon_quarter_regions, mean_biomass_quarter]).updateMask(combined_mask)



In [ ]:

# Function to preprocess images and create the unified data image
def create_unified_data(method, year):

    distance_to_nearest_mature = ee.Image(f"{data_folder}/{method}/distance_to_nearest_mature")
    cwd = ee.Image(f"{data_folder}/raw/cwd_chave").int16()
    sur_cover = ee.Image(f"{data_folder}/{method}/sur_cover")
    categorical = ee.Image(f"{data_folder}/categorical").select(["biome", "ecoreg", "protec", "indig"])
    topography = ee.Image("CSP/ERGo/1_0/Global/ALOS_landforms").rename("topography")
    soil = ee.Image(f"{data_folder}/soilgrids").select(['nitro', 'sand', 'phh2o'])  # Dropping multicollinear bands
    climate = ee.Image(f"{data_folder}/yearly_terraclim")

    secondary = ee.Image(f"{data_folder}/{method}/secondary")
    ages = secondary.select("age")

    distance_to_border_mask = ee.Image(f"{data_folder}/distance_to_border_mask").byte()
    one_hectare_mask = ee.Image(f"{data_folder}/{method}/one_hectare_mask").selfMask()

    base_name = f"{data_folder}/{method}/nearest_mature_biomass_"
    suffixes = range(1, 22)
    images = [ee.Image(f"{base_name}{suffix}") for suffix in suffixes]
    image_collection = ee.ImageCollection(images)
    nearest_mature = image_collection.mosaic().rename("nearest_mature_biomass")
    if method == "mapbiomas":
        nearest_mature = nearest_mature.reproject(scale = 1000, crs = 'EPSG:4326').rename("nearest_mature_biomass")

    # Combine all bands with additional masking conditions
    combined_mask = nearest_mature.mask().And(distance_to_border_mask).And(one_hectare_mask).And(ages_mask)#.And(GEDI_biomass.mask())

    unified_data = secondary.addBands([# GEDI_biomass, 
        nearest_mature, sur_cover, cwd, distance_to_nearest_mature, 
        categorical, soil, topography, climate
    ])

    if method == "mapbiomas":
        # fire = ee.Image(f"{data_folder}/{method}/ESA_fires")
        # unified_data = unified_data.addBands(fire.rename("ESA"))
        # suffix = "ESA_fire"
        
        if (aggregate_LU):
            suffix = "aggregated"
        else:
            suffix = "non_aggregated"

        if (year != 0):
            suffix += f"_{year}yr"
        else:
            suffix += "_all"

        # lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
        #             .select([f"classification_{year}" for year in config.range_1985_2020])
        #             .byte()
        #             .rename([str(year) for year in config.range_1985_2020]))
        # lulc = lulc.select("1985").eq(3).selfMask()
        land_use = ee.Image(f"{data_folder}/{method}/land_use_{suffix}")
        unified_data = unified_data.addBands([land_use])
        combined_mask = combined_mask.And(land_use.select("last_LU").mask())
    else:
        suffix = "eu"
        fire = ee.Image(f"{data_folder}/{method}/eu_mapbiomas_fires")
        mapbiomas_unburned_mask = ee.Image(f"{data_folder}/{method}/mapbiomas_unburned_mask")
        combined_mask = combined_mask.And(mapbiomas_unburned_mask)

        unified_data = unified_data.addBands([fire])

    unified_data = unified_data.updateMask(combined_mask)

    return unified_data, suffix

# Main Function to run tile-wise exports
def export_stratified(method, year):
    # Generate unified data image with selected layers
    unified_data, suffix = create_unified_data(method, year)
    # Filter properties to export
    to_remove = ['.geo', 'system:index']
    all_properties = unified_data.bandNames().getInfo()
    properties_to_export = [p for p in all_properties if p not in to_remove]

    unified_data_sampled = unified_data.stratifiedSample(numPoints = 15000, classBand = 'biome', geometries = False)
    
    # unified_data_sampled = unified_data.sample(region = roi, scale = 30, geometries = False)

    # Export task to Google Drive
    task = ee.batch.Export.table.toDrive(
        collection=unified_data_sampled,
        description=f'{suffix}',
        fileFormat='CSV',
        selectors=properties_to_export
    )
    task.start()



Started export task 1
Started export task 2
Started export task 3
Started export task 4
Started export task 5
Started export task 6
Started export task 7
Started export task 8
Started export task 9
Started export task 10
Started export task 11
Started export task 12
Started export task 13
Started export task 14
Started export task 15
Started export task 16
Started export task 17
Started export task 18
Started export task 19
Started export task 20
Started export task 21
Started export task 22
Started export task 1
Started export task 2
Started export task 3
Started export task 4
Started export task 5
Started export task 6
Started export task 7
Started export task 8
Started export task 9
Started export task 10
Started export task 11
Started export task 12
Started export task 13
Started export task 14
Started export task 15
Started export task 16
Started export task 17
Started export task 18
Started export task 19
Started export task 20
Started export task 21
Started export task 22


# Export full data for future prediction map

In [ ]:
# Main Function to run tile-wise exports
def export_tilewise(method, year):
    # Generate unified data image with selected layers
    unified_data, suffix = create_unified_data(method, year)
    # Filter properties to export
    to_remove = ['.geo', 'system:index']
    all_properties = unified_data.bandNames().getInfo()
    properties_to_export = [p for p in all_properties if p not in to_remove]

    # Load region of interest (ROI) and create a grid over the ROI
    grid = roi.coveringGrid("EPSG:4326", 1000000)
    tile_ids = grid.aggregate_array('system:index').getInfo()
    count = 0

    # Loop over IDs
    for feature_id in tile_ids:
        count = count + 1
        feat = grid.filter(ee.Filter.eq('system:index', feature_id))

        unified_data_sampled = unified_data.sample(region = feat.geometry(), scale = 100, geometries = False)

        # Export task to Google Drive
        task = ee.batch.Export.table.toDrive(
            collection=unified_data_sampled,
            description=f'{suffix}_{count}',
            fileFormat='CSV',
            selectors=properties_to_export
        )
        task.start()
        print(f'Started export task {count}')

In [ ]:
# to get the data from current agricultural land, to predict its regrowth potential:

# get all land use history for all plots that are currently agricultural land
# get the nearest mature biomass for each of these plots
# get the same predictors and run the model for these pixels.

canopy_height = ee.Image('users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1').clip(roi).rename('canopy_height')

if generate_lulc_pred:
    lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
            .select([f"classification_{year}" for year in config.range_1985_2020])
            .byte()
            .rename([str(year) for year in config.range_1985_2020]))

    age = remap_band(f"{last_year}", lulc)
